Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [293]:
from random import choices,randint ,random
import numpy as np
from copy import deepcopy
import networkx as nx
import matplotlib.pyplot as plt
from scipy.spatial.distance import hamming



# Copyright © 2023 Giovanni Squillero <giovanni.squillero@polito.it>
# https://github.com/squillero/computational-intelligence
# Free for personal or classroom use; see 'LICENSE.md' for details.

from abc import abstractmethod


class AbstractProblem:
    def __init__(self):
        self._calls = 0

    @property
    @abstractmethod
    def x(self):
        pass

    @property
    def calls(self):
        return self._calls

    @staticmethod
    def onemax(genome):
        return sum(bool(g) for g in genome)
     ##Slice the problem  with step sixe of self.x(problme size) and calculate fitness of each slice and then adad together 
     #slice overlaps if self.x>1
     #Fitness Call provide a result based on how similar are the slice based on onemax value
    def __call__(self, genome):
        self._calls += 1
        fitnesses = sorted((AbstractProblem.onemax(genome[s :: self.x]) for s in range(self.x)), reverse=True)
        val = sum(f for f in fitnesses if f == fitnesses[0]) - sum(
            f * (0.1 ** (k + 1)) for k, f in enumerate(f for f in fitnesses if f < fitnesses[0])
        )
        
        return (val / len(genome))
        
    


def make_problem(a):
    class Problem(AbstractProblem):
        @property
        @abstractmethod
        def x(self):
            return a

    return Problem()


In [294]:
POPULATION_SIZE = 50
OFFSPRING_SIZE = 100
LOCI = 1000
TOURNAMENT_SIZE = 5
MUTATION_PROBABILITY = 0.25
BIT_FLIP_PROBABILITY = 0.15
NUM_GENERATION = 1000
MUTATE_NEW_PROBABILITY = 0.15

In [295]:
class Genome:
    def __init__(self):
        self.genotype = choices([0, 1], k=LOCI)
        self.fitness = float("-inf")
        self.hamming_distance= float("-inf")



In [296]:
def new_offspring(parent1: Genome, parent2: Genome) -> Genome:
    if random() < MUTATION_PROBABILITY:
        return mutate(parent1)
    elif random()<MUTATE_NEW_PROBABILITY:
        return mutate_to_cover(parent1)
    return three_cut_xover(parent1, parent2)
    
def mutate(parent: Genome) -> Genome:
    new_offspring = deepcopy(parent)
    for i in range(LOCI-1):
        if random() < BIT_FLIP_PROBABILITY:
            new_offspring.genotype[i] = int(not new_offspring.genotype[i])
        
    return new_offspring

def mutate_to_cover(parent: Genome) -> Genome:
    new_offspring = deepcopy(parent)
    for i in range(LOCI-1):
        if(new_offspring.genotype[i])==0:
            if random() < 0.3:
                new_offspring.genotype[i] = int(not new_offspring.genotype[i])
        
    return new_offspring



def three_cut_xover(ind1: Genome, ind2: Genome) -> Genome:
    one_cut_point = randint(0, int((LOCI)*0.3))
    two_cut_point = randint(int((LOCI)*0.3), int((LOCI)*0.6))
    three_cut_point = randint(int((LOCI)*0.6), LOCI - 1)
  
    # Order the cut points
    cut_points = sorted([one_cut_point, two_cut_point, three_cut_point])
    
    new_ind = Genome()
    new_ind.genotype = (ind1.genotype[:cut_points[0]] +
                        ind2.genotype[cut_points[0]:cut_points[1]] +
                        ind1.genotype[cut_points[1]:cut_points[2]] +
                        ind2.genotype[cut_points[2]:])


    assert len(new_ind.genotype) == LOCI
    return new_ind

def hamming_distance(population: list[Genome]):
     

    # Calculate Hamming distances
    hamming_distances = np.zeros((len(population), len(population)))

    for i in range(len(population)):
        for j in range(i + 1, len(population)):
            distance = hamming(population[i].genotype, population[j].genotype)
            hamming_distances[i, j] = distance
            hamming_distances[j, i] = distance

    # Calculate the mean Hamming distance for each vector
    
   
      # Calculate mean Hamming distance for each genome
    mean_distances = np.mean(hamming_distances, axis=1)
    return mean_distances 
    #v1 v2 v3 -> agigungiamo al genoma

    # Find the indices of the genomes with the highest mean Hamming distance
    highest_mean_indices = np.argsort(mean_distances)[::-1]


    # Get the genomes with the highest mean Hamming distance
    highest_mean_genomes = [population[i] for i in highest_mean_indices]

    return highest_mean_genomes[:int(5)]

def hammin_distance2(population: list[Genome])->list[Genome]:
     

    # Calculate Hamming distances
    hamming_distances = np.zeros((len(population), len(population)))

    for i in range(len(population)):
        for j in range(i + 1, len(population)):
            distance = hamming(population[i].genotype, population[j].genotype)
            hamming_distances[i, j] = distance
            hamming_distances[j, i] = distance

    # Calculate the mean Hamming distance for each vector
    

      # Calculate mean Hamming distance for each genome
    mean_distances = np.mean(hamming_distances, axis=1)
    return mean_distances 
    # Find the indices of the genomes with the highest mean Hamming distance
    highest_mean_indices = np.argsort(mean_distances)[::-1]

   
   

    # Get the genomes with the highest mean Hamming distance
    highest_mean_genomes = [population[i] for i in highest_mean_indices]
    
    return highest_mean_genomes[:int(50)]
    
def sort_by_hamming_distance(population):
        tmp_population=deepcopy(population)
        new_means=hamming_distance(tmp_population)
        for  index,x in enumerate(tmp_population):
        
                x.hamming_distance=new_means[index]
        tmp_population.sort(key=lambda i: i.hamming_distance, reverse=True)
        return tmp_population

def sort_by_hamming_distance(population):
        tmp_population=deepcopy(population)
        new_means=hamming_distance(tmp_population)
        for  index,x in enumerate(tmp_population):
        
                x.hamming_distance=new_means[index]
        tmp_population.sort(key=lambda i: i.hamming_distance, reverse=True)
        return tmp_population


def evaluate_population(population):
    tmp_population=deepcopy(population)
    for p in tmp_population:
        p.fitness = fitness(p.genotype)
    return tmp_population
    


In [297]:
def graph(population: list[Genome]):
# Sample boolean vectors (replace these with your actual data)
   

    # Calculate Hamming distances
    hamming_distances = np.zeros((len(population), len(population)))

    for i in range(len(population)):
        for j in range(i + 1, len(population)):
            distance = hamming(population[i].genotype, population[j].genotype)
            hamming_distances[i, j] = distance
            hamming_distances[j, i] = distance

    # Calculate the mean Hamming distance for each vector
    mean_distances = np.mean(hamming_distances, axis=1)

    # Create a bar plot
    plt.figure(figsize=(10, 6))
    plt.bar(range(len(population)), mean_distances, color='skyblue', edgecolor='black')

    plt.title("Mean Hamming Distance for Each Vector")
    plt.xlabel("Vector Index")
    plt.ylabel("Mean Hamming Distance")
    plt.show()

In [350]:
fitness = make_problem(5)
    # starting pouplation of POPULATION_SIZE individuals
population = [Genome() for _ in range(POPULATION_SIZE)]

population=evaluate_population(population)

population.sort(key=lambda i: i.fitness, reverse=True)
#population=hammin_distance(population)
best_fitness = population[0].fitness
# for _ in range(NUM_GENERATION)
gen = 0
while  gen < 10:
        old_best_fitness = best_fitness
       
        
        for _ in range(OFFSPRING_SIZE):
               offspring = new_offspring(population[0],population[1])
               offspring.fitness=fitness(offspring.genotype)
               population.extend([offspring])
       
        population.sort(key=lambda i: i.fitness, reverse=True)
        #population=hammin_distance(population)
        population = population[:POPULATION_SIZE]
        best_fitness=population[0]
        #graph(population)
        gen += 1

print(f"{population[0].fitness:.2%}")
print(fitness.calls)

32.48%
1050


In [351]:
fitness = make_problem(5)
    # starting pouplation of POPULATION_SIZE individuals
population = [Genome() for _ in range(POPULATION_SIZE)]
population=evaluate_population(population)

population.sort(key=lambda i: i.fitness, reverse=True)
best_fitness = population[0].fitness
# for _ in range(NUM_GENERATION)
gen = 0
while gen < 10:
        offspring=[]
        ##create child
        for i in range(OFFSPRING_SIZE):
               child = new_offspring(population[0],population[1])
               offspring.append(child)
        #evaluate child by hamming_distance
        offspring=sort_by_hamming_distance(offspring)
        offspring=offspring[:10]
        offspring=evaluate_population(offspring)
        population.extend(offspring)
        population.sort(key=lambda i: i.fitness, reverse=True)
        population=population[:POPULATION_SIZE]
     
        best_fitness=population[0]
       
        gen += 1
print(gen)
print(f"{population[0].fitness:.2%}")
print(fitness.calls)


10
31.27%
150


In [300]:
fitness = make_problem(k)
fitness([1,1,1,1,0,0,0,0,0,0,0,0,0,0])
 1000//k  =0 

0.2857142857142857